In [1]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import sklearn
from scipy import stats
%matplotlib inline

In [9]:
'''
Here I will iteratively add the data to a single data frame. I have already
defined a paths.txt file which includes relative paths to all of my data.
Basically I will chop off the first two characters, './', and replace them with
a relative path to the directory in which the data is stored. Then it will 
iterate over every data file and append it to a dataframe.
'''
paths_file = 'eventPaths.txt'#path to paths file
relPathToPaths = '../data/'
relativePathToData = '../data/'#relative path to data folder
paths = open(paths_file, 'r')#open the paths file

#Creating dataframe

#now add each data file to the dataframe
for line in paths:
    path = str(relativePathToData+line[2:])
paths.close()#close the paths file

In [15]:
df = pd.read_csv('GL1999.TXT')

In [17]:
df.head()

,19990404,0,Sun,COL,NL,1,SDN,NL.1,1.1,8,...,Ruben Rivera,8.6,gomec001,Chris Gomez,6.4,ashba002.2,Andy Ashby.2,1.11,Unnamed: 159,Y
0,19990405,0,Mon,TBA,AL,1,BAL,AL,1,7,...,B.J. Surhoff,7,johnc002,Charles Johnson,2,reboj001,Jeff Reboulet,4,NaN,Y
1,19990405,0,Mon,BOS,AL,1,KCA,AL,1,5,...,Chad Kreuter,2,sancr001,Rey Sanchez,6,feblc001,Carlos Febles,4,NaN,Y
2,19990405,0,Mon,NYA,AL,1,OAK,AL,1,3,...,Eric Chavez,5,hinca001,A.J. Hinch,2,tejam001,Miguel Tejada,6,NaN,Y
3,19990405,0,Mon,CHA,AL,1,SEA,AL,1,8,...,John Mabry,7,davir002,Russ Davis,5,wilsd001,Dan Wilson,2,NaN,Y
4,19990405,0,Mon,DET,AL,1,TEX,AL,1,11,...,Lee Stevens,3,zeilt001,Todd Zeile,5,clayr001,Royce Clayton,6,NaN,Y
